In [61]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import StackingClassifier
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pylab import rcParams
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [62]:
df_train=pd.read_csv('Train_Set_90621.csv')
df_test=pd.read_csv('Test_Set_90621.csv')

In [3]:
df_test.shape

(83623, 28)

In [5]:
df_train.shape

(195118, 29)

In [6]:
df_train.isnull().sum()

Application ID                      0
Bank Masked                        80
Bank Type                          80
Approved_Timestamp                426
Name Masked                         1
Business Owner State                3
Business_Industry_Type_Code         0
Approved_Year                       0
New Business                       36
Term                                0
BankState                          81
Interest Rate                       0
Employees                           0
Gross Disbursed Amount              0
Term_years                          0
Jobs Retained                       0
Male to Female Employees Ratio      0
Expected Company Income             3
Funds available with company        0
Gross_Apprv_Amount                  0
Company Branch Code                 0
City or Rural                       0
Jobs Generated                      0
Carry-forward Credit              978
Documents Provided                552
Balance Left                        0
Amount Defau

In [7]:
df_test.isnull().sum()

Application ID                      0
Bank Masked                        48
Bank Type                          48
Approved_Timestamp                174
Name Masked                         0
Business Owner State                0
Business_Industry_Type_Code         0
Approved_Year                       0
New Business                       15
Term                                0
BankState                          48
Interest Rate                       0
Employees                           0
Gross Disbursed Amount              0
Term_years                          0
Jobs Retained                       0
Male to Female Employees Ratio      0
Expected Company Income             0
Funds available with company        0
Gross_Apprv_Amount                  0
Company Branch Code                 0
City or Rural                       0
Jobs Generated                      0
Carry-forward Credit              412
Documents Provided                247
Balance Left                        0
Amount Defau

In [3]:
df_train['Documents Provided']=df_train['Documents Provided'].fillna('N')
df_train['Carry-forward Credit']=df_train['Carry-forward Credit'].fillna('N')
df_train['BankState']=df_train['BankState'].fillna('UNKNOWN')
df_train['Bank Type']=df_train['Bank Type'].fillna('GOVT')
df_train['Business Owner State']=df_train['Business Owner State'].fillna('UNKNOWN')
df_train['New Business']=df_train['New Business'].fillna('0.0')
df_train['Expected Company Income']=df_train['Expected Company Income'].fillna(df_train['Expected Company Income'].mean())

In [4]:
df_test['Documents Provided']=df_test['Documents Provided'].fillna('N')
df_test['Carry-forward Credit']=df_test['Carry-forward Credit'].fillna('N')
df_test['BankState']=df_test['BankState'].fillna('UNKNOWN')
df_test['Bank Type']=df_test['Bank Type'].fillna('GOVT')
df_test['Business Owner State']=df_test['Business Owner State'].fillna('UNKNOWN')
df_test['New Business']=df_test['New Business'].fillna('0.0')
df_test['Expected Company Income']=df_test['Expected Company Income'].fillna(df_test['Expected Company Income'].mean())

Dropping Null values

In [63]:
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

In [64]:
df_train['Expected Company Income']=df_train['Expected Company Income'].astype('int64')

In [65]:
Q1=df_train.quantile(0.25)
Q3=df_train.quantile(0.75)
IQR=Q3-Q1
print(IQR)
#df = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
#df.shape

Application ID                       97588.50
Business_Industry_Type_Code         326620.00
Approved_Year                            6.00
New Business                             1.00
Term                                    60.00
Interest Rate                            5.00
Employees                               14.00
Gross Disbursed Amount            13830162.50
Term_years                               5.00
Jobs Retained                            4.00
Male to Female Employees Ratio          12.00
Expected Company Income            6700000.00
Funds available with company       1078811.75
Gross_Apprv_Amount                13440000.00
Company Branch Code                      0.00
City or Rural                            1.00
Jobs Generated                           1.00
Balance Left                             0.00
Amount Defaulted                         0.00
Final_Appved_Amount               10762500.00
Default_Status                           0.00
dtype: float64


Droping ID columns

In [66]:
df_train=df_train.drop(columns=['Application ID','Bank Masked','Name Masked','Approved_Timestamp'],axis=1)
df_test=df_test.drop(columns=['Application ID','Bank Masked','Name Masked','Approved_Timestamp'],axis=1)

In [67]:
df_cat_train=df_train.select_dtypes('object')
df_cat_test=df_test.select_dtypes('object')

In [68]:
df_cat_train=df_cat_train.drop(columns=['Business Owner State','BankState'],axis=0)
df_cat_test=df_cat_test.drop(columns=['Business Owner State','BankState'],axis=0)

In [69]:
df_cat_train_x1 = pd.get_dummies(df_cat_train, drop_first=True)
df_cat_test_x1 = pd.get_dummies(df_cat_test, drop_first=True)

In [70]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [71]:
df_train['Business Owner State']=le.fit_transform(df_train['Business Owner State'])
df_train['BankState']=le.fit_transform(df_train['BankState'])

df_test['Business Owner State']=le.fit_transform(df_test['Business Owner State'])
df_test['BankState']=le.fit_transform(df_test['BankState'])

In [72]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

In [73]:
XTrain=df_train.select_dtypes('int64')
XTest=df_test.select_dtypes('int64')

In [74]:
xtrain=pd.concat([XTrain,df_cat_train_x1],axis=1)
xtest=pd.concat([XTest,df_cat_test_x1],axis=1)

In [75]:
xi=xtrain.drop('Default_Status',axis=1)
xc=xtest

In [76]:
y=df_train['Default_Status']

In [77]:
xi.shape

(193042, 21)

In [78]:
from sklearn.preprocessing import StandardScaler
standardscalar=StandardScaler()

In [79]:
x_train_scaled=standardscalar.fit_transform(xi)
x_test_scaled=standardscalar.fit_transform(xc)

In [80]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression()

In [81]:
X_train, X_test, Y_train, Y_test = train_test_split(x_train_scaled, y,test_size=0.33,random_state=60)

In [82]:
logreg.fit(X_train,Y_train)
#Predicting variable for the test dataset
y_pred_train = logreg.predict(X_test)

In [83]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score, roc_curve

In [84]:
print(confusion_matrix(Y_test, y_pred_train))

[[51393   760]
 [ 9449  2102]]


In [85]:
print(metrics.classification_report(Y_test, y_pred_train))

              precision    recall  f1-score   support

           0       0.84      0.99      0.91     52153
           1       0.73      0.18      0.29     11551

    accuracy                           0.84     63704
   macro avg       0.79      0.58      0.60     63704
weighted avg       0.82      0.84      0.80     63704



In [31]:
from sklearn.tree import DecisionTreeClassifier

In [39]:
decision_tree = DecisionTreeClassifier(random_state=10,criterion='gini')#Default Gini
decision_tree = decision_tree.fit(X_train, Y_train)
y_pred = decision_tree.predict(X_test)
print("Accuracy:",accuracy_score(Y_test, y_pred))
#Gini-Accuracy: 0.884292980032651
#Entropy-Accuracy: 0.8839790280045209

Accuracy: 0.8839790280045209


In [41]:
print(confusion_matrix(Y_test, y_pred))

[[48259  3894]
 [ 3497  8054]]


In [42]:
print(metrics.classification_report(Y_test, y_pred_train))

              precision    recall  f1-score   support

           0       0.84      0.99      0.91     52153
           1       0.74      0.18      0.29     11551

    accuracy                           0.84     63704
   macro avg       0.79      0.58      0.60     63704
weighted avg       0.82      0.84      0.80     63704



In [86]:
from imblearn.over_sampling import SMOTE 
from sklearn.model_selection import cross_val_score

In [87]:
sm = SMOTE(random_state=123)
X_sm , y_sm = sm.fit_resample(x_train_scaled,y)

In [88]:
x_train , x_test , y_train ,y_test = train_test_split(X_sm,y_sm,test_size= 0.33)

In [89]:
logreg.fit(x_train,y_train)
#Predicting variable for the test dataset
y_pred_train = logreg.predict(x_test)

C:\Users\AS20188795\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [90]:
print("Accuracy:",accuracy_score(y_test, y_pred_train))

Accuracy: 0.7980363044207098


In [91]:
print(metrics.classification_report(y_test, y_pred_train))

              precision    recall  f1-score   support

           0       0.77      0.84      0.81     52105
           1       0.83      0.76      0.79     52290

    accuracy                           0.80    104395
   macro avg       0.80      0.80      0.80    104395
weighted avg       0.80      0.80      0.80    104395



In [92]:
decision_tree = decision_tree.fit(x_train, y_train)
y_pred = decision_tree.predict(x_test)
print("Accuracy:",accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

Accuracy: 0.9214521768283922
[[47844  4261]
 [ 3939 48351]]
              precision    recall  f1-score   support

           0       0.92      0.92      0.92     52105
           1       0.92      0.92      0.92     52290

    accuracy                           0.92    104395
   macro avg       0.92      0.92      0.92    104395
weighted avg       0.92      0.92      0.92    104395



In [58]:
scores = cross_val_score(estimator = decision_tree, 
                         X = x_train, 
                         y = y_train, 
                         cv = 10, 
                         scoring = 'accuracy')
print("Mean accuracy after 10 fold cross validation: ", round(scores.mean(), 2))

Mean accuracy after 10 fold cross validation:  0.92


In [95]:
rf_classification = RandomForestClassifier(n_estimators = 10, random_state = 10)
rf_model = rf_classification.fit(x_train, y_train)
y_pred = rf_model.predict(x_test)
print("Accuracy:",accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

Accuracy: 0.9429666171751521
              precision    recall  f1-score   support

           0       0.92      0.97      0.94     52105
           1       0.97      0.91      0.94     52290

    accuracy                           0.94    104395
   macro avg       0.94      0.94      0.94    104395
weighted avg       0.94      0.94      0.94    104395



In [97]:
base_learners = [('Decision Tree', DecisionTreeClassifier()),
                 ('Random Forest', RandomForestClassifier())]
stack_model_AdaBoost = StackingClassifier(estimators = base_learners, final_estimator = AdaBoostClassifier(random_state = 8))
stack_model_AdaBoost.fit(x_train, y_train)
y_pred_prob = stack_model_AdaBoost.predict_proba(x_test)[:, 1]
print('AUC score for the model with AdaBoost as final estimator:', roc_auc_score(x_test, y_pred_prob))

ValueError: continuous-multioutput format is not supported

In [ ]:
#tuned_paramaters = [{'criterion': ['entropy', 'gini'],
#                     'max_depth': range(2, 10),
#                     'min_samples_split': range(2,10)}]
#decision_tree_model = DecisionTreeClassifier(random_state = 10)
#tree_grid = GridSearchCV(estimator = decision_tree_model, 
#                         param_grid = tuned_paramaters, 
#                         cv = 5)
#tree_grid.fit(x_train, y_train)
#print('Best parameters for Decision Tree Classifier: ', tree_grid.best_params_, '\n')

In [44]:
import xgboost as xg
from xgboost import XGBClassifier

In [45]:
clf2 = xg.XGBClassifier(class_weight='balanced').fit(X_train, Y_train)

C:\Users\AS20188795\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:33:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "class_weight" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[08:33:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [46]:
from sklearn.utils import class_weight
#class_weight='balanced' uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data
class_weight.compute_class_weight('balanced', np.unique(Y_train), Y_train)

C:\Users\AS20188795\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=185850    0
78924     0
87194     0
179887    0
87099     0
         ..
194681    0
168008    0
174179    1
6543      0
133325    0
Name: Default_Status, Length: 130729, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


array([0.61054652, 2.76149134])

In [195]:
#Y_Test_Pred = clf2.predict(x_test_scaled)

In [78]:
generate_model_report(Y_test, Y_Test_Pred)

Accuracy =  0.9298003265101092
Precision =  0.9431576311506198
Recall =  0.6521513288892736
F1 Score =  0.7711127034496877


In [56]:
def generate_model_report(y_actual, y_predicted):
    print("Accuracy = " , accuracy_score(y_actual, y_predicted))
    print("Precision = " ,precision_score(y_actual, y_predicted))
    print("Recall = " ,recall_score(y_actual, y_predicted))
    print("F1 Score = " ,f1_score(y_actual, y_predicted))
    pass

In [59]:
best_feature=SelectKBest(score_func=chi2,k=10)
fit=best_feature.fit(xi,y)
df_score=pd.DataFrame(fit.scores_)
df_column=pd.DataFrame(xi.columns)
featurescore=pd.concat([df_column,df_score],axis=1)
featurescore.columns=['Specs','Score']
featurescore.nlargest(15,'Score')

,Specs,Score
16,Amount Defaulted,3.811323e+11
17,Final_Appved_Amount,3.693419e+10
11,Gross_Apprv_Amount,3.664286e+10
5,Gross Disbursed Amount,3.091477e+10
9,Expected Company Income,1.514742e+09
12,Company Branch Code,2.993971e+06
10,Funds available with company,5.791731e+05
2,Term,4.806896e+05
15,Balance Left,2.990999e+05
0,Business_Industry_Type_Code,1.299905e+05


In [65]:
xi_best=xi[['City or Rural','Term_years','Jobs Retained','Business_Industry_Type_Code','Balance Left','Term','Funds available with company','Company Branch Code','Expected Company Income','Gross Disbursed Amount','Gross_Apprv_Amount','Final_Appved_Amount','Amount Defaulted']]
xc_best=xc[['City or Rural','Term_years','Jobs Retained','Business_Industry_Type_Code','Balance Left','Term','Funds available with company','Company Branch Code','Expected Company Income','Gross Disbursed Amount','Gross_Apprv_Amount','Final_Appved_Amount','Amount Defaulted']]

In [14]:
#x=df_train[['Amount Defaulted','Final_Appved_Amount','Gross_Apprv_Amount','Gross Disbursed Amount','Company Branch Code','Funds available with company','Term','Balance Left','Jobs Generated','Employees','Jobs Retained','Business_Industry_Type_Code','Term_years','City or Rural','Male to Female Employees Ratio']]

,City or Rural,Term_years,Jobs Retained,Business_Industry_Type_Code,Balance Left,Term,Funds available with company,Company Branch Code,Expected Company Income,Gross Disbursed Amount,Gross_Apprv_Amount,Final_Appved_Amount,Amount Defaulted
0,1,18,18,525161,0,219,780590,25650,3500000,18550000,18550000,13912500,0
1,1,6,10,1231,0,71,1835637,0,1666667,3045000,2450000,1225000,2369500
2,2,7,10,322442,0,84,2373333,0,1333333,1750000,1750000,1487500,0
3,1,25,2,1231,0,300,2346786,1,333333,38850000,38850000,29137500,0
4,2,13,0,1231,0,153,2382995,1,333333,2800000,2800000,2380000,0


In [66]:
xi_train_scaled=standardscalar.fit_transform(xi)
xc_test_scaled=standardscalar.fit_transform(xc)

In [74]:
X_train, X_test, Y_train, Y_test = train_test_split(xi_train_scaled, y,test_size=0.33,random_state=60)
logreg.fit(X_train,Y_train)
y_pred_train = logreg.predict(xc_test_scaled)

C:\Users\AS20188795\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [73]:
print(metrics.classification_report(Y_test, y_pred_train))

              precision    recall  f1-score   support

           0       0.85      0.99      0.91     52821
           1       0.74      0.18      0.29     11568

    accuracy                           0.84     64389
   macro avg       0.79      0.58      0.60     64389
weighted avg       0.83      0.84      0.80     64389



In [80]:
base_learners = [('Decision Tree', DecisionTreeClassifier()),
                 ('Random Forest', RandomForestClassifier())]
stack_model_AdaBoost = StackingClassifier(estimators = base_learners, final_estimator = AdaBoostClassifier(random_state = 8))
stack_model_AdaBoost.fit(X_train, Y_train)
y_pred_prob = stack_model_AdaBoost.predict_proba(xc_test_scaled)[:, 1]

In [79]:
print('AUC score for the model with AdaBoost as final estimator:', roc_auc_score(Y_test, y_pred_prob))

AUC score for the model with AdaBoost as final estimator: 0.9411324074290102


In [35]:
submission = pd.read_csv('Sample_Sub.csv')

In [36]:
submission['Default_Status']=y_pred_train

In [37]:
submission.to_csv('Sample_Sub.csv', index=False)